In [81]:
import pandas as pd
import numpy as np
import os

# input and output paths
input_dir = "E:/K-MELLODDY-Project/data/ADMET_PK_Public_Dataset/01_06_TDC_embed_variance/Caco2_Wang/"

X_train_scaled = pd.read_csv(os.path.join(input_dir, "Caco2_Wang_train_fp_features_scaled.csv"))
for_y_train = pd.read_csv(os.path.join(input_dir, "Caco2_Wang_train_fp_features_raw.csv"))
y_train_fp = for_y_train.Label

X_val_scaled = pd.read_csv(os.path.join(input_dir, "Caco2_Wang_valid_fp_features_scaled.csv"))
for_y_val = pd.read_csv(os.path.join(input_dir, "Caco2_Wang_valid_fp_features_raw.csv"))
y_valid_fp = for_y_val.Label

X_test_scaled = pd.read_csv(os.path.join(input_dir, "Caco2_Wang_test_fp_features_scaled.csv"))
for_y_test = pd.read_csv(os.path.join(input_dir, "Caco2_Wang_test_fp_features_raw.csv"))
y_test_fp = for_y_test.Label

In [82]:
print(for_y_train.shape, X_train_scaled.shape, y_train_fp.shape)
print(for_y_val.shape, X_val_scaled.shape, y_valid_fp.shape)
print(for_y_test.shape, X_test_scaled.shape, y_test_fp.shape)

(637, 34) (637, 32) (637,)
(91, 34) (91, 32) (91,)
(182, 34) (182, 32) (182,)


In [83]:
X_train_fp_clean = X_train_scaled
X_valid_fp_clean = X_val_scaled
X_test_fp_clean = X_test_scaled

y_train_fp_clean = y_train_fp
y_valid_fp_clean = y_valid_fp
y_test_fp_clean = y_test_fp

### Stacking Model

In [84]:
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, StackingRegressor, HistGradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import make_regression # For a sample dataset
import os
import joblib

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

lgbm_model = LGBMRegressor(
    random_state=42,
    force_col_wise=True,
    device='cpu',          # enable GPU
    gpu_platform_id=0,     # CUDA platform ID
    gpu_device_id=0        # choose GPU 0 (RTX 5070 Ti)
)

xgb_model = XGBRegressor(
    random_state=42,
    eval_metric='rmse',
    tree_method='hist',     # use hist algorithm
    device='cuda'           # tell XGBoost to run on GPU
)

# Initialize Base Models for Regression
base_models = [
    ('et', ExtraTreesRegressor(random_state=42)),
    # ('hist', HistGradientBoostingRegressor(random_state=42)),
    ('rf', RandomForestRegressor(random_state=42)),
    ('lgbm', lgbm_model),
    ('xgb', xgb_model)
]

# Initialize Meta-Learner (Final Estimator)
meta_learner = Ridge(random_state=42)

# Create and Train the StackingRegressor
stacking_ensemble = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_learner,
    cv=10,       # Number of cross-validation folds for training the meta-learner
    n_jobs=-1    # Use all available cores for parallel processing
)

# Fit the model on the whole training set
stacking_ensemble.fit(X_train_fp_clean, y_train_fp_clean)

,estimators,"[('et', ...), ('rf', ...), ...]"
,final_estimator,Ridge(random_state=42)
,cv,10
,n_jobs,-1
,passthrough,False
,verbose,0
,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1


In [85]:
import os

save_path_output = input_dir + "Caco2_Wang_Outpus/"
save_path_plot = input_dir + "Caco2_Wang_Plots/"
os.makedirs(save_path_output, exist_ok=True)
os.makedirs(save_path_plot, exist_ok=True)
save_path_output, save_path_plot

('E:/K-MELLODDY-Project/data/ADMET_PK_Public_Dataset/01_06_TDC_embed_variance/Caco2_Wang/Caco2_Wang_Outpus/',
 'E:/K-MELLODDY-Project/data/ADMET_PK_Public_Dataset/01_06_TDC_embed_variance/Caco2_Wang/Caco2_Wang_Plots/')

In [86]:
import joblib

# Save the model
joblib.dump(stacking_ensemble, save_path_output + 'Caco2_Wang_stacking_model.pkl')

# Load the saved model
loaded_model = joblib.load(save_path_output + 'Caco2_Wang_stacking_model.pkl')

### Cross-Validation on Training Set

In [87]:
# import numpy as np
# import pandas as pd
# import os
# import joblib
# from sklearn.model_selection import cross_validate
# from sklearn.metrics import (
#     make_scorer, r2_score, mean_squared_error,
#     mean_absolute_error, mean_absolute_percentage_error
# )
# from scipy.stats import spearmanr

# # Define Spearman scorer
# def spearman_corr(y_true, y_pred):
#     return spearmanr(y_true, y_pred).correlation

# def smape(y_true, y_pred):
#     return np.mean(2.0 * np.abs(y_pred - y_true) /
#                    (np.abs(y_true) + np.abs(y_pred) + 1e-8)) * 100

# # Define regression scoring metrics
# scoring = {
#     'R2': make_scorer(r2_score),
#     'MAE': make_scorer(mean_absolute_error),
#     'RMSE': make_scorer(mean_squared_error, greater_is_better=False),  # will flip sign later
#     'SpearmanR': make_scorer(spearman_corr),
#     'MAPE': make_scorer(mean_absolute_percentage_error),
#     'SMAPE': make_scorer(smape, greater_is_better=False)
# }

# # Perform cross-validation
# cv_results = cross_validate(
#     estimator=loaded_model,
#     X=X_train_fp_clean,
#     y=y_train_fp_clean,
#     scoring=scoring,
#     cv=5,
#     return_train_score=False,
#     n_jobs=-1
# )

# # Prepare summary with mean ± std
# metrics_summary = {
#     'Metric': [],
#     'Mean ± Std': []
# }

# for metric in scoring.keys():
#     values = cv_results[f'test_{metric}']
#     if metric == 'RMSE':   # convert negative MSE → RMSE
#         values = np.sqrt(-values)
#     elif metric == 'SMAPE':  # SMAPE is set with greater_is_better=False
#         values = -values
    
#     mean_val = np.mean(values)
#     std_val = np.std(values)
#     metrics_summary['Metric'].append(metric.upper())
#     metrics_summary['Mean ± Std'].append(f"{mean_val:.4f} ± {std_val:.4f}")

# # Create DataFrame
# metrics_df_sf = pd.DataFrame(metrics_summary)

# # Save the model
# model_path = save_path_output + "Caco2_Wang_stacking_ensemble_model.pkl"
# joblib.dump(loaded_model, model_path)
# print(f"Model saved to: {model_path}")

# # Predictions (train set)
# y_pred_train = loaded_model.predict(X_train_fp_clean)

# # Build prediction DataFrame
# ids = pd.Series(np.arange(len(y_pred_train)), name="SampleID")
# pred_df = pd.DataFrame({
#     "ID": ids,
#     "True_Value": y_train_fp_clean.reset_index(drop=True),
#     "Predicted_Value": y_pred_train,
#     "Residual": y_train_fp_clean.reset_index(drop=True) - y_pred_train
# })

# # Save predictions
# pred_file = os.path.join(save_path_output, "Caco2_Wang_train_predicted_values.csv")
# pred_df.to_csv(pred_file, index=False)
# print(f"Predictions saved to: {pred_file}")

# # Save metrics summary
# metrics_file = os.path.join(save_path_output, "Caco2_Wang_train_cv_metrics_summary.csv")
# metrics_df_sf.to_csv(metrics_file, index=False)
# print(f"Metrics summary saved to: {metrics_file}")

# # Display metrics
# metrics_df_sf


### Evaluate on Validation and Test Sets

In [88]:
import numpy as np
import pandas as pd
import os
import joblib
from sklearn.metrics import (
    r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
)

from scipy.stats import spearmanr

def smape(y_true, y_pred):
    return np.mean(2.0 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred) + 1e-8)) * 100

# Define Spearman scorer
def spearman_corr(y_true, y_pred):
    return spearmanr(y_true, y_pred).correlation

# Define Evaluation Function for Regression
def evaluate_metrics(model, X, y, name="Set"):

    # Make predictions
    y_pred = model.predict(X)

    # Calculate regression metrics
    r2 = r2_score(y, y_pred)
    mae = mean_absolute_error(y, y_pred)
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    mape = mean_absolute_percentage_error(y, y_pred)
    spearman_corr, _ = spearmanr(y, y_pred)
    smap =  smape(y, y_pred)

    results = {
        "Set": name,
        "R2": r2,
        "MAE": mae,
        "RMSE": rmse,
        "SpearmanR": spearman_corr,
        "MAPE": mape,
        "SMAPE": smap        
    }

    # Return the metrics and the predictions
    return results, y_pred

# Replace extreme values with median
for col in X_valid_fp_clean.columns:
    col_vals = X_valid_fp_clean[col]
    X_valid_fp_clean[col] = np.where(
        np.abs(col_vals) > 1e10,
        np.median(col_vals[np.abs(col_vals) <= 1e10]),
        col_vals
    )

# Evaluate on Validation and Test
val_results, y_val_pred = evaluate_metrics(loaded_model, X_valid_fp_clean, y_valid_fp_clean, name="Validation")
test_results, y_test_pred = evaluate_metrics(loaded_model, X_test_fp_clean, y_test_fp_clean, name="Test")

# Save Evaluation Metrics
metrics_eval_df = pd.DataFrame([val_results, test_results])
print("Evaluation Metrics:\n", metrics_eval_df.to_string(index=False, float_format="{:.6f}".format))
metrics_eval_df.to_csv(save_path_output + "Caco2_Wang_validation_test_metrics_sf_scaled.csv", index=False,
                      float_format="%.6f")

# Save Actual vs Predicted
val_df = pd.DataFrame({
    'Set': 'Validation',
    'Actual': y_valid_fp_clean,
    'Predicted': y_val_pred
})

test_df = pd.DataFrame({
    'Set': 'Test',
    'Actual': y_test_fp_clean,
    'Predicted': y_test_pred
})

# Combine and save
results_df = pd.concat([val_df, test_df], ignore_index=True)
results_df.to_csv(save_path_output + "Caco2_Wang_actual_vs_predicted_sf_scaled.csv", index=False,
                 float_format="%.6f")

results_df

Evaluation Metrics:
        Set       R2      MAE     RMSE  SpearmanR     MAPE    SMAPE
Validation 0.440610 0.404363 0.510367   0.621573 0.078616 7.856528
      Test 0.412189 0.469287 0.611063   0.600076 0.089052 8.866207


,Set,Actual,Predicted
0,Validation,-5.722754,-4.935224
1,Validation,-4.699485,-5.400421
2,Validation,-5.647924,-5.472658
3,Validation,-5.190000,-4.508533
4,Validation,-6.000000,-5.558774
...,...,...,...
268,Test,-5.229574,-5.023439
269,Test,-5.000000,-5.577079
270,Test,-5.797940,-5.441377
271,Test,-4.480000,-4.642976


### Fold-wise Test Evaluation with Standard Deviation

In [89]:
import numpy as np
import pandas as pd
import os
import joblib
from sklearn.model_selection import KFold
from sklearn.metrics import (
    r2_score, mean_squared_error, mean_absolute_error,
    mean_absolute_percentage_error
)

def evaluate_with_std(model, X, y, n_splits=5):
    """
    Evaluates a model using K-fold cross-validation for regression and
    calculates the mean and standard deviation of metrics across folds.
    """
    # Use KFold for regression instead of StratifiedKFold
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    metrics_list = []
    all_preds = []

    # Ensure X and y are DataFrames/Series for consistent indexing
    X = pd.DataFrame(X).reset_index(drop=True)
    y = pd.Series(y).reset_index(drop=True)

    for fold, (train_idx, test_idx) in enumerate(kf.split(X, y), start=1):
        X_split, y_split = X.iloc[test_idx], y.iloc[test_idx]

        # Evaluate metrics and get predictions for this fold
        # The evaluate_metrics function for regression returns only two values
        metrics, y_pred = evaluate_metrics(model, X_split, y_split, name=f"Fold-{fold}")
        metrics_list.append(metrics)

        fold_df = pd.DataFrame({
            'Fold': fold,
            'Actual': y_split.values,
            'Predicted': y_pred
        })
        all_preds.append(fold_df)

    # Combine prediction results from all folds
    preds_df = pd.concat(all_preds, ignore_index=True)
    preds_df.to_csv(save_path_output + "Caco2_Wang_test_fold_predictions.csv", index=False)

    # Convert metrics list to DataFrame to calculate mean and std
    df_metrics = pd.DataFrame(metrics_list)

    # Calculate mean and std for each metric
    mean_metrics = df_metrics.mean(numeric_only=True)
    std_metrics = df_metrics.std(numeric_only=True)

    summary_df = pd.DataFrame({
        "Metric": mean_metrics.index,
        "Mean": mean_metrics.values,
        "Std": std_metrics.values
    })

    return summary_df

# Evaluate and save
test_metrics_summary = evaluate_with_std(loaded_model, X_test_fp_clean, y_test_fp_clean)
test_metrics_summary.to_csv(save_path_output + "Caco2_Wang_test_metrics_with_std.csv", index=False, float_format="%.6f")

test_metrics_summary

,Metric,Mean,Std
0,R2,0.357503,0.196442
1,MAE,0.469363,0.050693
2,RMSE,0.608483,0.064917
3,SpearmanR,0.591563,0.105455
4,MAPE,0.089061,0.005506
5,SMAPE,8.867130,0.712589


In [90]:
import pandas as pd

# Read the saved prediction file
preds_df = pd.read_csv(save_path_output + "Caco2_Wang_test_fold_predictions.csv")

# Initialize a dictionary to store predictions per fold
fold_columns = {}

# Loop through each fold
for fold in sorted(preds_df['Fold'].unique()):
    fold_name = f"Fold_{fold}"
    
    # Extract predicted probabilities for that fold
    fold_probs = preds_df[preds_df['Fold'] == fold]['Predicted'].reset_index(drop=True)
    
    # Store in dictionary
    fold_columns[fold_name] = fold_probs

# Create DataFrame with fold-wise predicted probabilities
folds_df = pd.DataFrame(fold_columns)

folds_df.to_csv(save_path_output + "Caco2_Wang_fold_predicted_probs_by_column.csv", index=False)

folds_df

,Fold_1,Fold_2,Fold_3,Fold_4,Fold_5
0,-4.450812,-4.837755,-5.486763,-5.245582,-5.203330
1,-5.554343,-4.734837,-5.164206,-5.135362,-5.057105
2,-5.219270,-4.521503,-5.255387,-4.827730,-5.532186
3,-4.830888,-5.290095,-4.307507,-4.363203,-4.781953
4,-5.396363,-4.941945,-5.582580,-5.234267,-4.771330
5,-5.337530,-4.357553,-4.355465,-6.161481,-6.754096
6,-5.248465,-4.654264,-5.540935,-5.578958,-5.924900
7,-5.826292,-4.840951,-5.614109,-5.208947,-5.695322
8,-5.557469,-5.088903,-4.940998,-5.315803,-5.125923
9,-3.920441,-5.252751,-4.687906,-4.847232,-4.841958


In [3]:
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, StackingRegressor, HistGradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import Ridge

# Instantiate models with defaults
et_default = ExtraTreesRegressor(random_state=42)

rf_default = RandomForestRegressor(random_state=42)

lgbm_default = LGBMRegressor(
    device='cpu',
    gpu_platform_id=0,      # Replace with actual platform ID
    gpu_device_id=0,        # Ensure this corresponds to RTX 5070 Ti
    random_state=42,
    force_col_wise=True
    # verbose=-1
)
xgb_default = XGBRegressor(
    tree_method='hist',
    device='cuda',
    predictor='auto',
    # gpu_id=0,
    eval_metric='rmse',
    random_state=42
)

ridgereg_default = Ridge(random_state=42)

# Print all default parameters
print("ExtraTreesClassifier defaults:\n", et_default.get_params(), "\n")
print("RandomForestClassifier defaults:\n", rf_default.get_params(), "\n")
print("LightGBMClassifier defaults:\n", lgbm_default.get_params(), "\n")
print("XGBClassifier defaults:\n", xgb_default.get_params(), "\n")
print("LogisticRegression defaults:\n", ridgereg_default.get_params(), "\n")

ExtraTreesClassifier defaults:
 {'bootstrap': False, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False} 

RandomForestClassifier defaults:
 {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False} 

LightGBMClassifier defaults:
 {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'l

In [8]:
import pprint
pprint.pprint(ridgereg_default.get_params())

{'alpha': 1.0,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': None,
 'positive': False,
 'random_state': 42,
 'solver': 'auto',
 'tol': 0.0001}
